In [2]:
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go
from stargazer.stargazer import Stargazer
import re
import plotly.graph_objects as go
from config_notebooks.config import set_wd
set_wd()

In [3]:
max_month_predict = 6
max_lag = 12
feature_list = ["pp", "pre", "er_cp", "gap","pi"]
alpha = .1

In [4]:
df = pd.read_excel("./data/data.xlsx", index_col=0)
df = df[:"2022-12"]
df

,pp,pi,gap,er_cp,pre
2005-01-31,508.020737,86.866026,1.154717,896.478093,1423.333333
2005-02-28,500.539941,87.907449,1.160477,872.658808,969.666667
2005-03-31,656.181802,86.226525,1.170062,860.907091,1730.000000
2005-04-30,939.064907,87.964450,1.149008,863.177404,620.000000
2005-05-31,1028.823820,89.117152,1.121609,854.918192,54.666667
...,...,...,...,...,...
2022-08-31,1988.408046,200.959296,2.149003,369.796103,146.666667
2022-09-30,2071.879245,204.883577,2.071033,368.751129,106.666667
2022-10-31,2339.045364,205.876822,2.022584,368.460742,543.333333
2022-11-30,1506.227931,203.938464,1.958168,373.953314,670.000000


In [5]:
def expand_df_max_lag(df_diff:pd.DataFrame, max_lag:int):
    last_date = df_diff.index.max()
    new_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), 
                            periods=max_lag, 
                            freq='MS')
    new_df = pd.DataFrame(index=new_dates)
    df_diff = pd.concat([df_diff, new_df])
    return df_diff

def create_lag_columns(df:pd.DataFrame, variable_name:str, max_lag:int):
    for lag in range(1, max_lag + 1):
        new_column_name = f"{variable_name}_lag_{lag}"
        df[new_column_name] = df[variable_name].shift(lag)
    return df

In [6]:
df = expand_df_max_lag(df, max_lag)
df = create_lag_columns(df, "er_cp", max_lag)
df = create_lag_columns(df, "pi", max_lag)
df = create_lag_columns(df, "pre", max_lag)
df = create_lag_columns(df, "pp", max_lag)
df = create_lag_columns(df, "gap", max_lag)

df_diff = df.diff()
df_diff

,pp,pi,gap,er_cp,pre,er_cp_lag_1,er_cp_lag_2,er_cp_lag_3,er_cp_lag_4,er_cp_lag_5,...,gap_lag_3,gap_lag_4,gap_lag_5,gap_lag_6,gap_lag_7,gap_lag_8,gap_lag_9,gap_lag_10,gap_lag_11,gap_lag_12
2005-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-28,-7.480796,1.041423,0.005760,-23.819285,-453.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-03-31,155.641861,-1.680925,0.009585,-11.751717,760.333333,-23.819285,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-30,282.883105,1.737925,-0.021054,2.270313,-1110.000000,-11.751717,-23.819285,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-31,89.758913,1.152702,-0.027399,-8.259212,-565.333333,2.270313,-11.751717,-23.819285,NaN,NaN,...,0.00576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.020233,-0.064416,-0.048449,-0.077971,-0.206273
2023-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.020233,-0.064416,-0.048449,-0.077971
2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.020233,-0.064416,-0.048449
2023-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.020233,-0.064416


## Optimum Lag
If the prediction is for one period: I should keep as a feature all those variables that has more than one lag. 

In [7]:
def obtain_features_prediction(df:pd.DataFrame, months_to_predict:int):
    columns_to_keep = []
    for column in df.columns:
        match = re.search(r'\d+$', column)
        if match:
            if int(match.group()) >= months_to_predict:
                columns_to_keep.append(column)
    df = df[columns_to_keep]

    return df

def limit_lags_feature(df:pd.DataFrame, variable:str, rezago_max:int):
    mask = df.columns.str.startswith(variable)
    df = df.iloc[:, mask]
    columns_to_drop = []
    for column in df.columns:
        match = re.search(r'\d+$', column)
        if match:
            if int(match.group()) > rezago_max:
                columns_to_drop.append(column)
    df = df.drop(columns_to_drop, axis=1)

    return df

In [8]:
#Feature calibration
for feature in feature_list:
    models_pp = []
    for lag in range(6, 13):
        df_train = df_diff[:-6].copy().dropna()
        y_train = df_train[feature]
        X_train = obtain_features_prediction(df_train, 6)
        X = limit_lags_feature(X_train, feature, lag)
        X = sm.add_constant(X)
        model = sm.OLS(y_train, X).fit()
        model = model.get_robustcov_results() #prueba
        models_pp.append(model)
        # print(model.summary())
    aic_values = [model.aic for model in models_pp]
    stargazer = Stargazer(models_pp)
    aic_notes = [f'Model {i+1} AIC: {aic}' for i, aic in enumerate(aic_values)]
    stargazer.add_custom_notes(aic_notes)
    tex_file = open( f"./output/calibration_mr/stargazer_calibracion_{feature}.tex", "w" ) #This will overwrite an existing file
    tex_file.write( stargazer.render_latex())
    tex_file.close()

In [9]:
def add_lag_calibration_aic(X_df:pd.DataFrame, features: list[str], y_df:pd.DataFrame, max_lag:int, months_to_predict:int):
    lag_calibration = pd.DataFrame(columns=features)
    for feature in features:
        for rezago in range(months_to_predict, max_lag + 1):
            X = limit_lags_feature(X_df, feature, rezago)
            X = sm.add_constant(X)
            model = sm.OLS(y_df, X).fit()
            model = model.get_robustcov_results() #prueba
            lag_calibration.loc[rezago,feature] = model.aic            
    return lag_calibration

def get_lowest_aic_lag(aic_results:pd.DataFrame):
    optimal_lag = {}
    for features in aic_results.columns:
        optimal_lag[features] = aic_results[features].sort_values(ascending=True).index[0]
    return optimal_lag

def get_aic_results(df_diff:pd.DataFrame,months_to_predict:int):
    df_train = df_diff[:-months_to_predict].copy().dropna()
    y_train = df_train["pp"]
    X_train = obtain_features_prediction(df_train, months_to_predict)

    aic_results = add_lag_calibration_aic(X_train, feature_list, y_train ,max_lag, months_to_predict)
    # print(months_to_predict, get_lowest_aic_lag(aic_results))
    aic_results.index.name = "Rezagos"
    return aic_results 
    
def write_aic_lags(df_diff:pd.DataFrame, max_month_predict:int):
    writer = pd.ExcelWriter("../data/calibration_rm/aic_lags.xlsx", engine="xlsxwriter")
    for months_to_predict in range(1,max_month_predict+1):
        aic_results = get_aic_results(df_diff,months_to_predict)
        aic_results.to_excel(writer, sheet_name=f"months_to_predict_{months_to_predict}")
    writer.close()
    
def get_optimal_lag_dict(df_diff:pd.DataFrame,max_month_predict:int):
    optimal_lag_dict = {}
    for months_to_predict in range(1,max_month_predict+1):
        aic_results = get_aic_results(df_diff, months_to_predict)
        optimal_lag_dict[months_to_predict] = get_lowest_aic_lag(aic_results)
    return optimal_lag_dict

In [10]:
optimal_lag_dict = get_optimal_lag_dict(df_diff, max_month_predict)
optimal_lag_dict

{1: {'pp': 12, 'pre': 9, 'er_cp': 1, 'gap': 4, 'pi': 1},
 2: {'pp': 12, 'pre': 9, 'er_cp': 2, 'gap': 4, 'pi': 2},
 3: {'pp': 12, 'pre': 8, 'er_cp': 3, 'gap': 4, 'pi': 5},
 4: {'pp': 12, 'pre': 7, 'er_cp': 4, 'gap': 4, 'pi': 5},
 5: {'pp': 12, 'pre': 12, 'er_cp': 5, 'gap': 5, 'pi': 5},
 6: {'pp': 12, 'pre': 12, 'er_cp': 6, 'gap': 11, 'pi': 6}}

## Modelo
1. Definir training y test. En test, se tiene que expandir la base de los lags
2. Correr la regresión con los lags óptimos y guardar el modelo

In [11]:
def get_lags_to_drop(columns:list[str], max_lag:int):
    lags_to_drop = []
    for column in columns:
        match = re.search(r'\d+$', column)
        if match:
            if int(match.group()) > max_lag:
                lags_to_drop.append(column)
    return lags_to_drop

In [12]:
def drop_lags_exog(features:list[str], X:pd.DataFrame, months_to_predict:int):
    for feature in features:
        lags_to_drop = X.columns[X.columns.str.startswith(feature)]
        lag_max = optimal_lag_dict[months_to_predict][feature]
        lags_to_drop = get_lags_to_drop(lags_to_drop, lag_max)  
        X = X.drop(lags_to_drop,axis=1)
    return X

In [13]:
models_mr = {}
for months_to_predict in range(1, max_month_predict + 1):
    df_train = df_diff.dropna()[:-months_to_predict]
    y_train = df_train.pp
    X_train = obtain_features_prediction(df_train, months_to_predict)
    X_train = drop_lags_exog(features = feature_list, X=X_train, months_to_predict = months_to_predict)
    # print(X_train)
    model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    model = model.get_robustcov_results() #prueba
    models_mr[months_to_predict] = model

In [14]:
tex_file = open( "./output/stargazer_rm_final_stargazer.tex", "w" ) #This will overwrite an existing file
stargazer = Stargazer([models_mr[6]])
tex_file.write( stargazer.render_latex())
# print(stargazer.title("Regresión múltiple: productos primarios"))

1495

In [15]:
tex_file = open("./output/stargazer_rm_final_py.tex","w")
tex_file.write(models_mr[6].summary().as_latex())

4881

In [16]:
models_mr[6].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     pp   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5.549
Date:                Sat, 25 Nov 2023   Prob (F-statistic):           1.75e-11
Time:                        23:07:48   Log-Likelihood:                -1345.7
No. Observations:                 197   AIC:                             2737.
Df Residuals:                     174   BIC:                             2813.
Df Model:                          22                                         
Covariance Type:                  HC1                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           6.5676     17.037      0.385      0.700     -27.059      40.194
er_cp_lag_6     0.5459      1.193      0.458      0.648      -1.809       2.901
pi_lag_6        1.8655      2.240      0.833      0.406      -2.555       6.286
pre_lag_6       0.0560      0.027      2.062      0.041       0.002       0.110
pre_lag_7       0.0061      0.035      0.173      0.863      -0.063       0.075
pre_lag_8      -0.0070      0.040     -0.175      0.861      -0.086       0.072
pre_lag_9      -0.0057      0.042     -0.138      0.891      -0.088       0.076
pre_lag_10     -0.0181      0.038     -0.476      0.635      -0.093       0.057
pre_lag_11     -0.0480      0.035     -1.366      0.174      -0.117       0.021
pre_lag_12     -0.0200      0.030     -0.668      0.505      -0.079       0.039
pp_lag_6       -0.2009      0.075     -2.686      0.008      -0.349      -0.053
pp_lag_7       -0.1542      0.072     -2.148      0.033      -0.296      -0.013
pp_lag_8       -0.0236      0.069     -0.343      0.732      -0.160       0.112
pp_lag_9        0.1002      0.068      1.484      0.140      -0.033       0.234
pp_lag_10       0.0273      0.069      0.395      0.694      -0.109       0.164
pp_lag_11       0.0488      0.071      0.688      0.493      -0.091       0.189
pp_lag_12       0.2748      0.071      3.846      0.000       0.134       0.416
gap_lag_6    -278.6136    230.490     -1.209      0.228    -733.530     176.303
gap_lag_7    -223.8711    220.548     -1.015      0.311    -659.164     211.422
gap_lag_8    -227.6302    294.068     -0.774      0.440    -808.030     352.769
gap_lag_9     400.2308    314.993      1.271      0.206    -221.469    1021.930
gap_lag_10   -163.5031    411.295     -0.398      0.691    -975.272     648.266
gap_lag_11    316.7828    273.701      1.157      0.249    -223.418     856.983
==============================================================================
Omnibus:                        6.045   Durbin-Watson:                   2.298
Prob(Omnibus):                  0.049   Jarque-Bera (JB):                6.020
Skew:                           0.321   Prob(JB):                       0.0493
Kurtosis:                       3.567   Cond. No.                     2.21e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
[2] The condition number is large, 2.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
models_predictions = {}
for months_to_predict in range(1,max_month_predict+1):
    y_test = df_diff[max_lag+1:-(max_lag - months_to_predict)]["pp"]
    X_test = obtain_features_prediction(df_diff, months_to_predict=months_to_predict)[max_lag+1:-(max_lag - months_to_predict)]
    X_test = drop_lags_exog(features = feature_list, X=X_test, months_to_predict=months_to_predict)
    model = models_mr[months_to_predict]
    prediction = model.get_prediction(sm.add_constant(X_test))   
    prediction = prediction.summary_frame(alpha = alpha)[["mean", "mean_ci_lower", "mean_ci_upper"]] #Confidence interval
    observ_vs_predict_df = pd.DataFrame({'observed': y_test, 
                                         'predicted': prediction["mean"],
                                         'predicted_lower': prediction["mean_ci_lower"],
                                         'predicted_upper': prediction["mean_ci_upper"],
                                         'pp_lag_1':df["pp_lag_1"]
                                         })
    observ_vs_predict_df["observed_reverted"] = observ_vs_predict_df["pp_lag_1"] + observ_vs_predict_df["observed"]
    observ_vs_predict_df["predicted_reverted"] = observ_vs_predict_df["pp_lag_1"] + observ_vs_predict_df["predicted"]
    observ_vs_predict_df["predicted_lower_reverted"] = observ_vs_predict_df["pp_lag_1"] + observ_vs_predict_df["predicted_lower"]
    observ_vs_predict_df["predicted_upper_reverted"] = observ_vs_predict_df["pp_lag_1"] + observ_vs_predict_df["predicted_upper"]
    observ_vs_predict_df = observ_vs_predict_df.dropna(subset=["predicted"])
    
    def get_revertion(observ_vs_predict_df: pd.DataFrame, variable:str):
        observ_vs_predict_df[f'prev_{variable}_reverted'] = observ_vs_predict_df[f"{variable}_reverted"].shift(1)
        observ_vs_predict_df['sum'] = observ_vs_predict_df[f'prev_{variable}_reverted'].add(observ_vs_predict_df[variable])
        observ_vs_predict_df[f"{variable}_reverted"] = observ_vs_predict_df[f"{variable}_reverted"].fillna(observ_vs_predict_df['sum'])
        observ_vs_predict_df = observ_vs_predict_df.drop([f'prev_{variable}_reverted', 'sum'],axis=1)
        return observ_vs_predict_df
    
    while observ_vs_predict_df['predicted_reverted'].isnull().any():
        observ_vs_predict_df = get_revertion(observ_vs_predict_df, variable="predicted")
        observ_vs_predict_df = get_revertion(observ_vs_predict_df, variable="predicted_lower")
        observ_vs_predict_df = get_revertion(observ_vs_predict_df, variable="predicted_upper")
    models_predictions[months_to_predict] = observ_vs_predict_df

In [18]:
models_predictions[6][-20:] #Problema con el mes: día 1

,observed,predicted,predicted_lower,predicted_upper,pp_lag_1,observed_reverted,predicted_reverted,predicted_lower_reverted,predicted_upper_reverted
2021-11-30,-507.902805,-290.187860,-446.160196,-134.215525,1826.193733,1318.290929,1536.005873,1380.033538,1691.978208
2021-12-31,434.133249,-71.204411,-208.307910,65.899089,1318.290929,1752.424178,1247.086518,1109.983018,1384.190018
2022-01-31,129.630687,27.294281,-103.146611,157.735173,1752.424178,1882.054864,1779.718459,1649.277567,1910.159350
2022-02-28,-50.345857,-112.391968,-230.831265,6.047329,1882.054864,1831.709007,1769.662896,1651.223599,1888.102193
2022-03-31,251.076815,153.617668,28.118717,279.116619,1831.709007,2082.785822,1985.326675,1859.827724,2110.825626
2022-04-30,212.692357,339.052854,225.132961,452.972748,2082.785822,2295.478179,2421.838676,2307.918783,2535.758570
2022-05-31,-180.554102,278.415361,125.338631,431.492090,2295.478179,2114.924077,2573.893540,2420.816811,2726.970270
2022-06-30,-79.714692,-49.748252,-188.464950,88.968447,2114.924077,2035.209385,2065.175825,1926.459127,2203.892524
2022-07-31,396.209023,17.834413,-149.798330,185.467157,2035.209385,2431.418408,2053.043799,1885.411055,2220.676542
2022-08-31,-443.010362,97.679432,-85.937208,281.296072,2431.418408,1988.408046,2529.097840,2345.481200,2712.714480


In [29]:
def plot_prediccion_rm(models_predictions:list, months_to_predict:int = max_month_predict):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = models_predictions[months_to_predict].index, y = models_predictions[months_to_predict].observed_reverted, name = "Observado"))
    fig.add_trace(go.Scatter(x = models_predictions[months_to_predict].index, y = models_predictions[months_to_predict].predicted_reverted, name = "Predicción"))
    fig.add_trace(go.Scatter(x = models_predictions[months_to_predict].index[-months_to_predict:], y = models_predictions[months_to_predict].predicted_lower_reverted[-months_to_predict:], name = "Predicción_lower"))
    fig.add_trace(go.Scatter(x = models_predictions[months_to_predict].index[-months_to_predict:], y = models_predictions[months_to_predict].predicted_upper_reverted[-months_to_predict:], name = "Predicción_upper", fill='tonexty'))
    fig.add_vline(x = models_predictions[months_to_predict].index[-months_to_predict], line_width=3, line_dash="dash", line_color="green")
    fig.update_layout(template = None, 
                      separators = ",.",
                      height = 300,
                    #   title_text = f"Predicción de las exportaciones de los siguientes {months_to_predict} meses en base a una regresión múltiple<br><sup>Nivel de confianza: {1-alpha:.0%}",
                      font_family = "georgia",
                      margin = dict(l=30, t=0, b=30))
    return fig

plot_prediccion_rm(models_predictions, months_to_predict = 6)

In [30]:
plot_prediccion_rm(models_predictions, months_to_predict = 6).write_image("./output/Prediccion_rm_plot.pdf")